In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
from tensorflow.keras.models import load_model

In [2]:
### Load the trained model , scaker pickle , one hot 
model = load_model('model.h5')

### laod the encoder and scaler
with open('label_encoder_geo.pkl','rb') as file:
    label_encorder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encorder_gender=pickle.load(file)
    
with open('sscaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
# One hot encode the Geo
geo_encoder = label_encorder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder, columns= label_encorder_geo.get_feature_names_out(['Geography']))

D:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [5]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
input_df = pd.concat([input_df.reset_index(drop =True),geo_encoder_df], axis = 1)


In [8]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
input_df = input_df.drop(columns = ["Geography"])

In [10]:
### Encode the categorical value gender
input_df['Gender'] = label_encorder_gender.transform(input_df['Gender'])

In [11]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
# scaling the input data
input_scaler = scaler.transform(input_df)

In [13]:
input_scaler

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
prediction = model.predict(input_scaler)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[0.03674467]], dtype=float32)

In [15]:
if prediction > 0.5:
    print("Mpst likely to churn")
else:
    print("Not likely to churn")

Not likely to churn
